In [25]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [26]:
#读取数据
file_path={'sst2':'SST2/train.tsv'}
df_list = []
for source, file in file_path.items():
     df=pd.read_csv(file,names=['sentence','label'],sep='\t')
     df_list.append(df)
df = pd.concat(df_list)
print(df.head())

                                            sentence  label
0  a stirring , funny and finally transporting re...      1
1  apparently reassembled from the cutting room f...      0
2  they presume their audience wo n't sit still f...      0
3  this is a visually stunning rumination on love...      1
4  jonathan parker 's bartleby should have been t...      1


In [27]:
print(type(df))
print(df['sentence'])
#print(dl['splitset_label'])

<class 'pandas.core.frame.DataFrame'>
0       a stirring , funny and finally transporting re...
1       apparently reassembled from the cutting room f...
2       they presume their audience wo n't sit still f...
3       this is a visually stunning rumination on love...
4       jonathan parker 's bartleby should have been t...
                              ...                        
6915    painful , horrifying and oppressively tragic ,...
6916    take care is nicely performed by a quintet of ...
6917    the script covers huge , heavy topics in a bla...
6918    a seriously bad film with seriously warped log...
6919    a deliciously nonsensical comedy about a city ...
Name: sentence, Length: 6920, dtype: object


In [28]:
import re
import string
from collections import OrderedDict

def cleanInput(input):
    input = re.sub('\n+', " ", input)
    input = re.sub('\[[0-9]*\]', "", input)
    input = re.sub(' +', " ", input)
    input = bytes(input, "UTF-8")
    input = input.decode("ascii", "ignore")
    cleanInput = []
    input = input.split(' ')
    for item in input:
        item = item.strip(string.punctuation)
        if len(item) > 1 or (item.lower() == 'a' or item.lower() == 'i'):
            cleanInput.append(item)
    return cleanInput

def getNgrams(input, n):
    input = cleanInput(input)
    #print(input)
    output = dict()
    lis = []
    for j in range(n+1):
        if j ==0:
            continue
        for i in range(len(input)-j+1):
            newNGram = " ".join(input[i:i+j])
            lis.append(newNGram)
            if newNGram in output:
                output[newNGram] += 1
            else:
                output[newNGram] = 1
    return output,lis


    # for i in range(len(input)-n+1):
    #     newNGram = " ".join(input[i:i+n])
    #     lis.append(newNGram)
    #     if newNGram in output:
    #         output[newNGram] += 1
    #     else:
    #         output[newNGram] = 1
    # return output,lis

In [29]:
output,lis = getNgrams("I'd like swimming i",2)
print(type(output))
print(lis)

<class 'dict'>
["I'd", 'like', 'swimming', 'i', "I'd like", 'like swimming', 'swimming i']


In [30]:
# coding:utf-8
import numpy as np
from scipy import sparse

class MyCountVectorizer():
    def __init__(self, ngram_range=1, pass_stop=True):
        self.pass_stop = pass_stop # 提供停止词滤除功能，可禁止
        self.ngram_range = ngram_range #提供ngram的功能
        
    def fit(self, data):
        n_data = data #使用ngram的部分数据
        # data = map(lambda x:str(x).split(" "), data)
        self.elements_ = set()
        # if self.ngram_range ==1:
        #     for line in data:
        #         for x in line:
        #             if self.pass_stop:
        #                 if len(x)==1:
        #                     continue
        #             self.elements_.add(x)
        # else:
        for line in n_data:
            dic,li = getNgrams(line,self.ngram_range)
            for x in li:
                # if self.pass_stop:
                #     if len(x)==1:
                #         continue
                self.elements_.add(x)
        # 原元素
        self.elements_ = np.sort(list(self.elements_))
        # 编码
        self.labels_ = np.arange(len(self.elements_)).astype(int)
        # 生成字典
        self.dict_ = {}
        for i in range(len(self.elements_)):
            self.dict_[str(self.elements_[i])] = self.labels_[i]

    def transform(self, data):
        rows = []
        cols = []
        n_data = data
        data = map(lambda x:str(x).split(" "), data)
        l_data = list(data)
        #print(data)
        #print(n_data.size)

        # for i in range(len(l_data)):
        #     for x in l_data[i]:
        #         if self.pass_stop:
        #             if len(x)==1:
        #                 continue
        #         rows.append(i)
        #         print("这一次的输出i是%d,这一次输出的x是%d",i,x)
        #         cols.append(self.dict_[x])
        index =0;
        count = 0;
        cn =0;
        for line in n_data:
            dic,li = getNgrams(line,self.ngram_range)
            for x in li:
                if self.pass_stop:
                    if len(x)==1:
                        continue
               #print("这一次的输出i是%d,这一次输出的x是%d",index,x)
                
                if x in self.dict_:
                    rows.append(index)
                    cols.append(self.dict_[x])
                    cn = cn+1
                else:
                    count = count+1
                    # self.dict_[x]
                    # cols.append(self.dict_[x])
                    #print(x)
            
            index = index + 1 


        vals = np.ones((len(rows),)).astype(int)
        print("cn是",cn)
        print("count是",count)
        #return sparse.csr_matrix((vals, (rows, cols)), shape=(len(l_data), len(self.labels_)))
        return sparse.csr_matrix((vals, (rows, cols)), shape=(index, len(self.labels_)))

In [31]:
sentence = df['sentence'].values[:1000]
#print(sentence[:100])
#print(type(sentence))
y = df['label'].values[:1000]
#划分处数据集
sentence_train,sentence_test,y_train,y_test=train_test_split(sentence,y,test_size=0.25,random_state=1000)
#print(type(y_train))#<class 'numpy.ndarray'>
vectorize = MyCountVectorizer(ngram_range=2)
vectorize.fit(sentence)
X_train = vectorize.transform(sentence_train)
X_test = vectorize.transform(sentence_test)
print(X_train.shape)
#print(X_train.shape)#(5190, 203644)
#print(type(X_train))#'scipy.sparse.csr.csr_matrix'
xtra = X_train.toarray()
xtes = X_test.toarray()
#print(type(xt))#<class 'numpy.ndarray'>
#源代码
# sentence = df['sentence'].values[:1000]
# #print(sentence[:100])
# #print(type(sentence))
# y = df['label'].values[:1000]
# #划分处数据集
# sentence_train,sentence_test,y_train,y_test=train_test_split(sentence,y,test_size=0.25,random_state=1000)
# #print(type(y_train))#<class 'numpy.ndarray'>
# vectorize = CountVectorizer(min_df=0,lowercase=False,ngram_range=(1,4))
# vectorize.fit(sentence_train)
# X_train = vectorize.transform(sentence_train)
# X_test = vectorize.transform(sentence_test)
# #print(X_train.shape)#(5190, 203644)
# #print(type(X_train))#'scipy.sparse.csr.csr_matrix'
# #xt = X_train.toarray()
# #print(type(xt))#<class 'numpy.ndarray'>

cn是 24691
count是 0
cn是 8419
count是 0
(750, 17526)


In [32]:
#查看更换后的数据
#print(xt.shape[0])
# s = set()
# for i in range(xt.shape[0]):
#     for j in range(xt.shape[1]):
#         if xt[i][j] > 1:
#             #print(xt[i][j])
#             s.add(xt[i][j])
# print(s)

In [33]:
classifier = LogisticRegression()
classifier.fit(X_train,y_train)
print(type(classifier))
score = classifier.score(X_test,y_test)
print(score)#自己的是076

<class 'sklearn.linear_model._logistic.LogisticRegression'>
0.676


In [34]:
import time
import numpy as np
from collections import defaultdict

In [35]:
class maxEnt:
    '''
    最大熵类
    '''
    def __init__(self, trainDataList, trainLabelList, testDataList, testLabelList):
        '''
        各参数初始化
        '''
        self.trainDataList = trainDataList          #训练数据集
        self.trainLabelList = trainLabelList        #训练标签集
        self.testDataList = testDataList            #测试数据集
        self.testLabelList = testLabelList          #测试标签集
        self.featureNum = len(trainDataList[0])     #特征数量
        self.N = len(trainDataList)                 #总训练集长度
        self.n = 0                                  #训练集中（xi，y）对数量
        self.M = 10000                                 #
        self.fixy = self.calc_fixy()                #所有(x, y)对出现的次数
        self.w = [0] * self.n                       #Pw(y|x)中的w
        self.xy2idDict, self.id2xyDict = self.createSearchDict()        #(x, y)->id和id->(x, y)的搜索字典
        self.Ep_xy = self.calcEp_xy()               #Ep_xy期望值
    def calcEpxy(self):
        '''
        计算特征函数f(x, y)关于模型P(Y|X)与经验分布P_(X, Y)的期望值（P后带下划线“_”表示P上方的横线
        程序中部分下划线表示“|”，部分表示上方横线，请根据具体公式自行判断,）
        即“6.2.2 最大熵模型的定义”中第二个期望（83页最上方的期望）
        :return:
        '''
        #初始化期望存放列表，对于每一个xy对都有一个期望
        #这里的x是单个的特征，不是一个样本的全部特征。例如x={x1，x2，x3.....，xk}，实际上是（x1，y），（x2，y），。。。
        #但是在存放过程中需要将不同特诊的分开存放，李航的书可能是为了公式的泛化性高一点，所以没有对这部分提及
        #具体可以看我的博客，里面有详细介绍  www.pkudodo.com
        Epxy = [0] * self.n
        #对于每一个样本进行遍历
        for i in range(self.N):
            #初始化公式中的P(y|x)列表
            Pwxy = [0] * 2
            #计算P(y = 0 } X)
            #注：程序中X表示是一个样本的全部特征，x表示单个特征，这里是全部特征的一个样本
            Pwxy[0] = self.calcPwy_x(self.trainDataList[i], 0)
            #计算P(y = 1 } X)
            Pwxy[1] = self.calcPwy_x(self.trainDataList[i], 1)
            for feature in range(self.featureNum):
                for y in range(2):
                    if (self.trainDataList[i][feature], y) in self.fixy[feature]:
                        id = self.xy2idDict[feature][(self.trainDataList[i][feature], y)]
                        Epxy[id] += (1 / self.N) * Pwxy[y]
        return Epxy
    def calcEp_xy(self):
        '''
        计算特征函数f(x, y)关于经验分布P_(x, y)的期望值（下划线表示P上方的横线，
        同理Ep_xy中的“_”也表示p上方的横线）
        即“6.2.2 最大熵的定义”中第一个期望（82页最下方那个式子）
        :return: 计算得到的Ep_xy
        '''
        #初始化Ep_xy列表，长度为n
        Ep_xy = [0] * self.n
        #遍历每一个特征
        for feature in range(self.featureNum):
            #遍历每个特征中的(x, y)对
            for (x, y) in self.fixy[feature]:
                #获得其id
                id = self.xy2idDict[feature][(x, y)]
                #将计算得到的Ep_xy写入对应的位置中
                #fixy中存放所有对在训练集中出现过的次数，处于训练集总长度N就是概率了
                Ep_xy[id] = self.fixy[feature][(x, y)] / self.N
        #返回期望
        return Ep_xy
    def createSearchDict(self):
        '''
        创建查询字典
        xy2idDict：通过(x,y)对找到其id,所有出现过的xy对都有一个id
        id2xyDict：通过id找到对应的(x,y)对
        '''
        #设置xy搜多id字典
        #这里的x指的是单个的特征，而不是某个样本，因此将特征存入字典时也需要存入这是第几个特征
        #这一信息，这是为了后续的方便，否则会乱套。
        #比如说一个样本X = (0, 1, 1) label =(1)
        #生成的标签对有(0, 1), (1, 1), (1, 1)，三个(x，y)对并不能判断属于哪个特征的，后续就没法往下写
        #不可能通过(1, 1)就能找到对应的id，因为对于(1, 1),字典中有多重映射
        #所以在生成字典的时总共生成了特征数个字典，例如在mnist中样本有784维特征，所以生成784个字典，属于
        #不同特征的xy存入不同特征内的字典中，使其不会混淆
        xy2idDict = [{} for i in range(self.featureNum)]
        #初始化id到xy对的字典。因为id与(x，y)的指向是唯一的，所以可以使用一个字典
        id2xyDict = {}
        #设置缩影，其实就是最后的id
        index = 0
        #对特征进行遍历
        for feature in range(self.featureNum):
            #对出现过的每一个(x, y)对进行遍历
            #fixy：内部存放特征数目个字典，对于遍历的每一个特征，单独读取对应字典内的(x, y)对
            for (x, y) in self.fixy[feature]:
                #将该(x, y)对存入字典中，要注意存入时通过[feature]指定了存入哪个特征内部的字典
                #同时将index作为该对的id号
                xy2idDict[feature][(x, y)] = index
                #同时在id->xy字典中写入id号，val为(x, y)对
                id2xyDict[index] = (x, y)
                #id加一
                index += 1
        #返回创建的两个字典
        return xy2idDict, id2xyDict
    def calc_fixy(self):
        '''
        计算(x, y)在训练集中出现过的次数
        :return:
        '''
        #建立特征数目个字典，属于不同特征的(x, y)对存入不同的字典中，保证不被混淆
        fixyDict = [defaultdict(int) for i in range(self.featureNum)]
        #遍历训练集中所有样本
        for i in range(len(self.trainDataList)):
            #遍历样本中所有特征
            for j in range(self.featureNum):
                #将出现过的(x, y)对放入字典中并计数值加1
                fixyDict[j][(self.trainDataList[i][j], self.trainLabelList[i])] += 1
        #对整个大字典进行计数，判断去重后还有多少(x, y)对，写入n
        for i in fixyDict:
            self.n += len(i)
        #返回大字典
        return fixyDict
    def calcPwy_x(self, X, y):
        '''
        计算“6.23 最大熵模型的学习” 式6.22
        :param X: 要计算的样本X（一个包含全部特征的样本）
        :param y: 该样本的标签
        :return: 计算得到的Pw(Y|X)
        '''
        #分子
        numerator = 0
        #分母
        Z = 0
        #对每个特征进行遍历
        for i in range(self.featureNum):
            #如果该(xi,y)对在训练集中出现过
            if (X[i], y) in self.xy2idDict[i]:
                #在xy->id字典中指定当前特征i，以及(x, y)对：(X[i], y)，读取其id
                index = self.xy2idDict[i][(X[i], y)]
                #分子是wi和fi(x，y)的连乘再求和，最后指数
                #由于当(x, y)存在时fi(x，y)为1，因为xy对肯定存在，所以直接就是1
                #对于分子来说，就是n个wi累加，最后再指数就可以了
                #因为有n个w，所以通过id将w与xy绑定，前文的两个搜索字典中的id就是用在这里
                numerator += self.w[index]
            #同时计算其他一种标签y时候的分子，下面的z并不是全部的分母，再加上上式的分子以后
            #才是完整的分母，即z = z + numerator
            if (X[i], 1-y) in self.xy2idDict[i]:
                #原理与上式相同
                index = self.xy2idDict[i][(X[i], 1-y)]
                Z += self.w[index]
        #计算分子的指数
        numerator = np.exp(numerator)
        #计算分母的z
        Z = np.exp(Z) + numerator
        #返回Pw(y|x)
        return numerator / Z
    def maxEntropyTrain(self, iter = 144):#300
        #设置迭代次数寻找最优解
        for i in range(iter):
            #单次迭代起始时间点
            iterStart = time.time()
            #计算“6.2.3 最大熵模型的学习”中的第二个期望（83页最上方哪个）
            Epxy = self.calcEpxy()
            #使用的是IIS，所以设置sigma列表
            sigmaList = [0] * self.n
            #对于所有的n进行一次遍历
            for j in range(self.n):
                #依据“6.3.1 改进的迭代尺度法” 式6.34计算
                sigmaList[j] = (1 / self.M) * np.log(self.Ep_xy[j] / Epxy[j])
            #按照算法6.1步骤二中的（b）更新w
            self.w = [self.w[i] + sigmaList[i] for i in range(self.n)]
            #单次迭代结束
            iterEnd = time.time()
            #打印运行时长信息
            print('iter:%d:%d, time:%d'%(i, iter, iterEnd - iterStart))
    def predict(self, X):
        '''
        预测标签
        :param X:要预测的样本
        :return: 预测值
        '''
        #因为y只有0和1，所有建立两个长度的概率列表
        result = [0] * 2
        #循环计算两个概率
        for i in range(2):
            #计算样本x的标签为i的概率
            result[i] = self.calcPwy_x(X, i)
        #返回标签
        #max(result)：找到result中最大的那个概率值
        #result.index(max(result))：通过最大的那个概率值再找到其索引，索引是0就返回0，1就返回1
        return result.index(max(result))
    def test(self):
        '''
        对测试集进行测试
        :return:
        '''
        #错误值计数
        errorCnt = 0
        #对测试集中所有样本进行遍历
        for i in range(len(self.testDataList)):
            #预测该样本对应的标签
            result = self.predict(self.testDataList[i])
            #如果错误，计数值加1
            if result != self.testLabelList[i]:   errorCnt += 1
        #返回准确率
        return 1 - errorCnt / len(self.testDataList)

In [36]:
start = time.time()
#先不二值化 看看效果如何
maxEnt = maxEnt(xtra, y_train, xtes, y_test)
#maxEnt = maxEnt(xtra[:100], y_train[:100], xtes[:100], y_test[:100])
#开始训练
print('start to train')
maxEnt.maxEntropyTrain()
#开始测试
print('start to test')
accuracy = maxEnt.test()
print('the accuracy is:', accuracy)
# 打印时间
print('time span:', time.time() - start)

start to train
iter:0:144, time:105
iter:1:144, time:103
iter:2:144, time:107
iter:3:144, time:106
iter:4:144, time:106
iter:5:144, time:104
iter:6:144, time:106
iter:7:144, time:107
iter:8:144, time:105
iter:9:144, time:107
iter:10:144, time:107
iter:11:144, time:105
iter:12:144, time:106
iter:13:144, time:107
iter:14:144, time:105
iter:15:144, time:106
iter:16:144, time:106
iter:17:144, time:106
iter:18:144, time:105
iter:19:144, time:105
iter:20:144, time:104
iter:21:144, time:103
iter:22:144, time:106
iter:23:144, time:106
iter:24:144, time:107
iter:25:144, time:108
iter:26:144, time:109
iter:27:144, time:108
iter:28:144, time:109
iter:29:144, time:109
iter:30:144, time:109
iter:31:144, time:108
iter:32:144, time:108
iter:33:144, time:107
iter:34:144, time:107
iter:35:144, time:107
iter:36:144, time:110
iter:37:144, time:107
iter:38:144, time:107
iter:39:144, time:109
iter:40:144, time:107
iter:41:144, time:109
iter:42:144, time:109
iter:43:144, time:107
iter:44:144, time:107
iter:

In [37]:
res = maxEnt.predict(xtes[0])
print(res)
print(y_test[0])

1
1
